# Data Header Testing Notebook

In the cell below, fill out the list of MIRISim output files you want to test. Then run the notebook to find out what's missing!

In [1]:
from pathlib import Path

In [31]:
data_files = [Path("/Users/jaguilar/Projects/mirisim/vega_data/Vega_MIRISim_Coron_Simulations_v1/Vega_F1550C_Roll1/20210625_153310_mirisim_vega_stardisk_star16Jy_roll1_v1/det_images/det_image_seq1_MIRIMAGE_F1550Cexp1.fits"),
              Path("/Users/jaguilar/Projects/mirisim/vega_data/Vega_MIRISim_Coron_Simulations_v1/Vega_F1550C_Roll2/20210625_155253_mirisim_vega_stardisk_star16Jy_roll2_v1/det_images/det_image_seq1_MIRIMAGE_F1550Cexp1.fits"),
             ]
psf_path = Path("/Users/jaguilar/Projects/mirisim/vega_data/Vega_MIRISim_Coron_Simulations_v1/Vega_PSF_Ref_F1550C_piHer/20210703_piHer_PSFref_all-positions_v1/")
psf_files = [list((p / "det_images").glob("*fits"))[0] for p in psf_path.glob("*SGD[0-9]*")]
data_files += psf_files

In [32]:
data_files

[PosixPath('/Users/jaguilar/Projects/mirisim/vega_data/Vega_MIRISim_Coron_Simulations_v1/Vega_F1550C_Roll1/20210625_153310_mirisim_vega_stardisk_star16Jy_roll1_v1/det_images/det_image_seq1_MIRIMAGE_F1550Cexp1.fits'),
 PosixPath('/Users/jaguilar/Projects/mirisim/vega_data/Vega_MIRISim_Coron_Simulations_v1/Vega_F1550C_Roll2/20210625_155253_mirisim_vega_stardisk_star16Jy_roll2_v1/det_images/det_image_seq1_MIRIMAGE_F1550Cexp1.fits'),
 PosixPath('/Users/jaguilar/Projects/mirisim/vega_data/Vega_MIRISim_Coron_Simulations_v1/Vega_PSF_Ref_F1550C_piHer/20210703_piHer_PSFref_all-positions_v1/20210702_141821_mirisim_piHer_PSFref_star19Jy_SGD5_v1/det_images/det_image_seq1_MIRIMAGE_F1550Cexp1.fits'),
 PosixPath('/Users/jaguilar/Projects/mirisim/vega_data/Vega_MIRISim_Coron_Simulations_v1/Vega_PSF_Ref_F1550C_piHer/20210703_piHer_PSFref_all-positions_v1/20210702_135810_mirisim_piHer_PSFref_star19Jy_SGD1_v1/det_images/det_image_seq1_MIRIMAGE_F1550Cexp1.fits'),
 PosixPath('/Users/jaguilar/Projects/miris

## Tests

Draft of helpful summary information:
- a list of which headers are and aren't included
- a list of which keywords are and aren't included, by header
- a list of whick keywords have improper values

In [33]:
from importlib import reload
import test_miri_coron_kwds as tmck

## Test for existence

In [34]:
reload(tmck)

TK = tmck.TestKwds(data_files)

In [35]:
for file in TK.mirisim_files:
    print(f"\n{TK.active_file.name}\n")
    TK.active_file = file
    TK.test_all_headers_kwds_exist()


det_image_seq1_MIRIMAGE_F1550Cexp1.fits

Active file changed; loading new headers
Testing PRIMARY
45/211 expected keywords were found.
The following keywords *do* exist:
     ACT_ID   CCCSTATE   CORONMSK   DATE-OBS
   TARG_DEC   DETECTOR   DURATION     EXPEND
   EXPOSURE   EFFEXPTM     EXTEND   FASTAXIS
   FILENAME   FILETYPE     FILTER     TFRAME
     TGROUP   GROUPGAP   DATAMODL    NFRAMES
    NGROUPS      NINTS    NRESETS   NSAMPLES
     OBS_ID   OBSERVTN     ORIGIN    PROGRAM
    TARG_RA   READPATT    TSAMPLE     SEQ_ID
     SIMPLE   SLOWAXIS   EXPSTART   TELESCOP
   TIMEUNIT   VISITGRP   VISIT_ID      VISIT
   SUBSIZE1   SUBSTRT1   SUBSIZE2   SUBSTRT2
   ZEROFRAM                                 

The following keywords *do not* exist:
     ACT_ID   CCCSTATE   CORONMSK   DATE-OBS
   TARG_DEC   DETECTOR   DURATION     EXPEND
   EXPOSURE   EFFEXPTM     EXTEND   FASTAXIS
   FILENAME   FILETYPE     FILTER     TFRAME
     TGROUP   GROUPGAP   DATAMODL    NFRAMES
    NGROUPS      NINTS  

## Keyword lists

Any keyword that has a list of values; check that its assigned value is in the list

In [36]:
reload(tmck)
TK = tmck.TestKwds(data_files)

In [37]:
TK.test_kwd_enum_values('PRIMARY')

5 keywords tested
5/5 keywords are OK: 
     FILTER   DETECTOR   CCCSTATE   READPATT
   CORONMSK                                 


In [38]:
TK.test_all_headers_kwds_enum()

Testing PRIMARY
5 keywords tested
5/5 keywords are OK: 
     FILTER   DETECTOR   CCCSTATE   READPATT
   CORONMSK                                 


Testing SCI
0 keywords tested


Testing PIXELDQ
0 keywords tested


Testing REFOUT
0 keywords tested


Testing ASDF
0 keywords tested




In [30]:
for file in TK.mirisim_files:
    print(f"\n{TK.active_file.name}\n")
    TK.active_file = file
    TK.test_all_headers(TK.test_kwd_enum_values)


det_image_seq1_MIRIMAGE_F1550Cexp1.fits

Active file changed; loading new headers
Testing PRIMARY
5 keywords tested
4/5 keywords are OK: 
     FILTER   DETECTOR   CCCSTATE   READPATT
1/5 keywords have a value not in the allowed list: 
CORONMSK has value FQPM_1550
but must be one of
LYOT LYOT_2300 4QPM 4QPM_1065 4QPM_1140 4QPM_1550


Testing SCI
0 keywords tested


Testing PIXELDQ
0 keywords tested


Testing REFOUT
0 keywords tested


Testing ASDF
0 keywords tested



det_image_seq1_MIRIMAGE_F1550Cexp1.fits

Active file changed; loading new headers
Testing PRIMARY
5 keywords tested
4/5 keywords are OK: 
     FILTER   DETECTOR   CCCSTATE   READPATT
1/5 keywords have a value not in the allowed list: 
CORONMSK has value FQPM_1550
but must be one of
LYOT LYOT_2300 4QPM 4QPM_1065 4QPM_1140 4QPM_1550


Testing SCI
0 keywords tested


Testing PIXELDQ
0 keywords tested


Testing REFOUT
0 keywords tested


Testing ASDF
0 keywords tested



det_image_seq1_MIRIMAGE_F1550Cexp1.fits

Active file ch

## Test for type

OK, now do a test for each kind of type

Can you do a 2-level groupby for enum and type?
For enum, you need to group by the result of a function that returns True if list and False if not a list.

In [ ]:
reload(tmck)
TK = tmck.TestKwds(data_files)

In [ ]:
for file in TK.mirisim_files:
    print(f"\n{TK.active_file.name}\n")
    TK.active_file = file
    TK.test_all_headers(TK.test_kwd_type)